In [2]:
import sys
sys.path.append('../')

from src.sudoku import Sudoku
from src.time_utils import Timer
import src.solver as solver
# import src.tester as tester
import statistics as stat
from scipy import stats as sci_stats

In [3]:
import src.func_sudoku as func_sudoku
import src.func_solver as func_solver

In [4]:
import json

In [5]:
%load_ext autoreload
%autoreload 2

# Tests

In [6]:
with open('../tests/easy_tests.json', 'r') as easy_tests:
    easy_boards = json.loads(easy_tests.read())

In [7]:
with open('../tests/medium_tests.json', 'r') as medium_tests:
    medium_boards = json.loads(medium_tests.read())

In [8]:
with open('../tests/hard_tests.json', 'r') as hard_tests:
    hard_boards = json.loads(hard_tests.read())

# Testing 1

In [28]:
game_stats = []

In [29]:
boards = medium_boards
# boards = hard_boards
# boards = easy_boards

In [34]:
test_boards = {
    'easy': easy_boards,
    'medium': medium_boards,
    'hard': hard_boards,
}

In [36]:
for level, boards in test_boards.items():
    print(level)
    level_stats = []
    
    for board in boards:
        new_game = func_sudoku.create_sudoku(board)

        complete, finished_game = func_solver.human_mixed_priority_backtracking_heap(new_game, lambda x: func_solver.aggregate_pure_solver(x, [
            func_solver.numberwise, func_solver.cellwise
        ]))
        if not complete:
            print(board)
            break
        level_stats.append((finished_game['guesses'], finished_game['removes']))
    print(len(level_stats))
    print('avg guesses', sum([x[0] for x in level_stats]) / len(level_stats))
    print('avg removes', sum([x[1] for x in level_stats]) / len(level_stats))

easy
40
avg guesses 2.7
avg removes 0.925
medium
40
avg guesses 6.675
avg removes 8.9
hard
40
avg guesses 11.3
avg removes 10.175


# Heuristic Generation

In [202]:
easy_board = tester.get_sugoku_board('easy')

easy_game = func_sudoku.create_sudoku(easy_board)


AttributeError: module 'src.func_solver' has no attribute 'numberwise_backtracking'

# Testing

## Test 1

Paired t-test. The goal is to compare two methods to see if the metric we choose (time, guesses) differs.

### Example

In [6]:
trials = 100

In [7]:
hard_board = tester.get_sugoku_board('hard')

In [8]:
hard_game = Sudoku(hard_board)

In [9]:
solvers = [solver.cellwise_mixed_priority_backtracking_manual, 
           solver.priority_backtracking_manual]

In [10]:
timers = [Timer(f.__name__) for f in solvers]

In [11]:
for timer, sudo_solver in zip(timers, solvers):
    solver_guesses = []
    for _ in range(trials):
        game = Sudoku(hard_board)
        empty = game.number_empty()
        timer.start()
        done, guesses = sudo_solver(game)
        timer.stop(verbose=False)
        if not done:
            print('unsolved')
        else:
            solver_guesses.append(guesses / empty)

    print("{:50}: Avg Number of Percentage Guess: {:.5f}".format(sudo_solver.__name__, stat.mean(solver_guesses)))
for timer in timers:
    timer.summary()

cellwise_mixed_priority_backtracking_manual       : Avg Number of Percentage Guess: 0.38889
priority_backtracking_manual                      : Avg Number of Percentage Guess: 1.09259
------------------------------
Name               : cellwise_mixed_priority_backtracking_manual
Count              : 100
Total Time         : 0.94361s
Average Time       : 0.00944s
Standard Deviation : 0.00075s
------------------------------
------------------------------
Name               : priority_backtracking_manual  
Count              : 100
Total Time         : 0.78537s
Average Time       : 0.00785s
Standard Deviation : 0.00031s
------------------------------


In [12]:
sci_stats.ttest_rel(timers[0].times, timers[1].times)

Ttest_relResult(statistic=20.979153069208085, pvalue=3.2466782660166373e-38)